<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tensorboard使用流程" data-toc-modified-id="Tensorboard使用流程-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tensorboard使用流程</a></span></li><li><span><a href="#Tensorboard的数据形式" data-toc-modified-id="Tensorboard的数据形式-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tensorboard的数据形式</a></span></li><li><span><a href="#通过命名空间美化计算图" data-toc-modified-id="通过命名空间美化计算图-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>通过命名空间美化计算图</a></span></li><li><span><a href="#tf.summary.merge_all" data-toc-modified-id="tf.summary.merge_all-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>tf.summary.merge_all</a></span></li><li><span><a href="#tf.summary.FileWriter" data-toc-modified-id="tf.summary.FileWriter-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>tf.summary.FileWriter</a></span></li><li><span><a href="#完整示例-(一)" data-toc-modified-id="完整示例-(一)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>完整示例 (一)</a></span></li><li><span><a href="#完整示例-(二)" data-toc-modified-id="完整示例-(二)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>完整示例 (二)</a></span></li></ul></div>

## Tensorboard使用流程
[Link1](http://blog.csdn.net/mzpmzk/article/details/77914941)

* 添加记录节点 `tf.summary.scalar/ tf.summary.image / tf.summary.histogram`
* 汇总记录节点 `merged = tf.summary.merge_all()`
* 运行汇总节点 `summary = ses.run(merged) # 得到汇总结果` 
* 日志书写器实例化 `summary_writer = tf.summary.FileWriter(logdir, graph=sess.graph) # 实例化的同时传入graph将当前计算图写入日子`
* 调用日志书写器对象 `summary_writer.add_summary(summary, global_step=i)`
* 关闭日志书写器对象 `summary_writer.close()`


## Tensorboard的数据形式
***

* 标量Scalars            (使用`tf.summary.scalar`记录)
* 图片Images
* 音频Audio
* 计算图Graph           （使用`tf.summary.image`记录）
* 数据分布Distribution  （使用`tf.summary.distribution`记录）
* 直方图Histograms      （使用`tf.summary.histogram`记录）
* 嵌入向量Embeddings

```python
# 标量
tf.summary.scalar('loss', loss)
# 备注： 只能展示一个实数值, 也就是一般用来显示准确度、损失等，不能用来展示训练好的权重矩阵或向量
# 参数：tf.summary.scalar(name, tensor, collections=None, family=None)
* name:   A name for the generated node. Will also serve as the series name in TensorBoard.
* tensor:  A real numeric Tensor containing a single value.
* collections:  Optional list of graph collections keys. The new summary op is added to these collections. Defaults to `[GraphKeys.SUMMARIES]`.
* family:   Optional; if provided, used as the prefix of the summary tag name, which controls the tab name used for display on Tensorboard.

# 直方图
tf.summary.histogram('bias', bias)
# 备注： 只能展示tensor的分布，并不能表示tensor的值
# 参数： tf.sumary.histogram(name, values, collections=None, family=None):
* name: string，表示在tensorboard显示的名称
* values: tensor，一般是多维tensor
* collections: 表示将这个类加入到哪个集合中
* family: 类似命名前缀。一般只要在定义图的时候进行了命名空间定义,如name_scope、variable_scope，就不需要这family了

# 图像
tf.summary.image('img', tf.reshape(images, [-1, 28, 28, 1]), 10)
# 备注: reshape里面的四个参数分别代表[图像数, 每幅图的高度, 每幅图的宽度, 每幅图的通道数]，-1表示根据实际数据（在这里是images）进行动态计算; 第三个参数"10"表示最多展示十幅图

# 音频
tf.summary.audio('audio', audio, sampling_frequency)
# 备注: audio是一个三维或者二维tensor，含义是[音频数, 每个音频的帧数, 每个音频的通道数]或者[音频数, 每个音频的帧数]。sampling_frequency从名字就可以看出来了，就是音频的采样率
```


## 通过命名空间美化计算图

* 使用命名空间使可视化效果图更有层次性，使得神经网络的整体结构不会被过多的细节所淹没
* 同一个命名空间下的所有节点会被缩略成一个节点，只有顶层命名空间中的节点才会被显示在 TensorBoard 可视化效果图上
* 可通过`tf.name_scope()`或者`tf.variable_scope()`来实现

In [1]:
import tensorflow as tf

In [ ]:
'''
tf.variable_scope可以让变量有相同的命名，包括tf.get_variable得到的变量，还有tf.Variable的变量
tf.name_scope可以让变量有相同的命名，只是限于tf.Variable的变量 

可以看出，对于tf.Variable()函数，两者的使用情况都一样；
而tf.get_variable()函数，它不受name_scope约束，已经声明过的变量就不能再声明了
'''

import tensorflow as tf

with tf.variable_scope('V1'):
    a1 = tf.get_variable(name='a1', shape=[1], initializer=tf.constant_initializer(1))
    a2 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')
with tf.variable_scope('V2'):
    a3 = tf.get_variable(name='a1', shape=[1], initializer=tf.constant_initializer(1))
    a4 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')
  
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print a1.name       # V1/a1:0
    print a2.name       # V1/a2:0
    print a3.name       # V2/a1:0
    print a4.name       # V2/a1:0
    
    
with tf.name_scope('V1'):
    # a1 = tf.get_variable(name='a1', shape=[1], initializer=tf.constant_initializer(1))
    a2 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')
with tf.name_scope('V2'):
    # a3 = tf.get_variable(name='a1', shape=[1], initializer=tf.constant_initializer(1))
    a4 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')
  
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # print a1.name    # 报错: Variable a1 already exists, disallowed. 
    print a2.name      # V1/a2:0
    # print a3.name    # 报错: Variable a1 already exists, disallowed. 
    print a4.name      # V2/a2:
    

## tf.summary.merge_all
***

tensorflow 中的 operations 并不会立即执行计算, 除非显示调用run去执行该operation, 或者它被其他的需要run的operation所依赖。但是一些常用的summary operations其实并不被其他节点依赖，因此，需要特地去运行所有的summary节点。但是，一份程序下来可能有超多这样的summary 节点，要手动一个一个去启动自然是及其繁琐的，可以使用`tf.summary.merge_all`去将所有summary节点合并成一个节点，只要运行这个节点，就能产生所有之前设置的summary data。

## tf.summary.FileWriter
***

使用`tf.summary.FileWriter`可以将运行后输出的数据都保存到本地磁盘中

参数说明：
```python
def __init__(self,
           logdir,
           graph=None,
           max_queue=10,
           flush_secs=120,
           graph_def=None,
           filename_suffix=None,
           session=None):
        
Args:
  logdir:   A string. Directory where event file will be written.
  graph:    A `Graph` object, such as `sess.graph`.
  max_queue:   Integer. Size of the queue for pending events and summaries.
  flush_secs:   Number. How often, in seconds, to flush the pending events and summaries to disk.
  graph_def:  DEPRECATED: Use the `graph` argument instead.
  filename_suffix:  A string. Every event files name is suffixed with suffix
  session:  A `tf.Session` object. See details above.        
```

## 完整示例 (一)

In [ ]:
# -*- coding: UTF-8 -*

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 100
display_step = 1
logs_path = 'tensorflow_logs'

# Network Parameters
n_hidden_1 = 20 # 1st layer number of features
n_hidden_2 = 40 # 2nd layer number of features
n_input = 784   # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# 使用tf.summary.scalar记录标量
# 使用tf.summary.histogram记录数据的直方图
# 使用tf.summary.distribution记录数据的分布图
# 使用tf.summary.image记录图像数据

# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Create a summary to visualize the first layer ReLU activation
    tf.summary.histogram("relu1", layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Create another summary to visualize the second layer ReLU activation
    tf.summary.histogram("relu2", layer_2)
    # Output layer
    out_layer = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    return out_layer

# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'w3': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W3')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'b3': tf.Variable(tf.random_normal([n_classes]), name='b3')
}

# Encapsulating all ops into scopes, making Tensorboard's Graph
# Visualization more convenient
with tf.name_scope('Model'):
    # Build model
    pred = multilayer_perceptron(x, weights, biases)

with tf.name_scope('Loss'):
    # Softmax Cross entropy (cost function)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Op to calculate every variable gradient
    grads = tf.gradients(loss, tf.trainable_variables())
    grads = list(zip(grads, tf.trainable_variables()))
    # Op to update all variables according to their gradient
    apply_grads = optimizer.apply_gradients(grads_and_vars=grads)

with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Create summaries to visualize weights
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
# Summarize all gradients
for grad, var in grads:
    tf.summary.histogram(var.name + '/gradient', grad)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([apply_grads, loss, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

    print("Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")

## 完整示例 (二)

In [ ]:
# -*- coding: utf-8 -*-

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


# 准备训练数据，假设其分布大致符合 y = 1.2x + 0.0
n_train_samples = 200
X_train = np.linspace(-5, 5, n_train_samples)
Y_train = 1.2*X_train + np.random.uniform(-1.0, 1.0, n_train_samples)  # 加一点随机扰动


# 准备验证数据，用于验证模型的好坏
n_test_samples = 50
X_test = np.linspace(-5, 5, n_test_samples)
Y_test = 1.2*X_test


# 参数学习算法相关变量设置
learning_rate = 0.01
batch_size = 20
summary_dir = 'logs'

print '~~~~~~~~~~开始设计计算图~~~~~~~~'

# 使用 placeholder 将训练数据/验证数据送入网络进行训练/验证
# shape=None 表示形状由送入的张量的形状来确定
with tf.name_scope('Input'):
    X = tf.placeholder(dtype=tf.float32, shape=None, name='X')
    Y = tf.placeholder(dtype=tf.float32, shape=None, name='Y')


# 决策函数(参数初始化)
with tf.name_scope('Inference'):
    W = tf.Variable(initial_value=tf.truncated_normal(shape=[1]), name='weight')
    b = tf.Variable(initial_value=tf.truncated_normal(shape=[1]), name='bias')
    Y_pred = tf.multiply(X, W) + b


# 损失函数(MSE)
with tf.name_scope('Loss'):
    loss = tf.reduce_mean(tf.square(Y_pred - Y), name='loss')
    tf.summary.scalar('loss', loss)


# 参数学习算法(Mini-batch SGD)
with tf.name_scope('Optimization'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)


# 初始化所有变量
init = tf.global_variables_initializer()


# 汇总记录节点
merge = tf.summary.merge_all()


# 开启会话，进行训练
with tf.Session() as sess:
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logdir=summary_dir, graph=sess.graph)

    for i in range(201):
        j = np.random.randint(0, 10)  # 总共200训练数据，分十份[0, 9]
        X_batch = X_train[batch_size*j: batch_size*(j+1)]
        Y_batch = Y_train[batch_size*j: batch_size*(j+1)]

        _, summary, train_loss, W_pred, b_pred = sess.run([optimizer, merge, loss, W, b], feed_dict={X: X_batch, Y: Y_batch})
        test_loss = sess.run(loss, feed_dict={X: X_test, Y: Y_test})

        # 将所有日志写入文件
        summary_writer.add_summary(summary, global_step=i)
        print('step:{}, losses:{}, test_loss:{}, w_pred:{}, b_pred:{}'.format(i, train_loss, test_loss, W_pred[0], b_pred[0]))

        if i == 200:
            # plot the results
            plt.plot(X_train, Y_train, 'bo', label='Train data')
            plt.plot(X_test, Y_test, 'gx', label='Test data')
            plt.plot(X_train, X_train * W_pred + b_pred, 'r', label='Predicted data')
            plt.legend()
            plt.show()

    summary_writer.close()